# Biodiversity Assesment of the Phoenix Metropolitan Area

A geographic assesment of biodiversity in Maricopa County, utilizing the biodiversity intactness index (BII).

## Background
Add summary...

## Project Setup

Libraries

In [14]:
import numpy as np
import geopandas as gpd
import rioxarray as rioxr
import matplotlib.pyplot as plt

from shapely.geometry import Polygon

from pystac_client import Client
import planetary_computer
from IPython.display import Image

### Loading Data
BII will be accessed through the MPC STAC catalog, search for `io-biodiversity`

In [25]:
# access mpc catalog
catalog = Client.open(
    "https://planetarycomputer.microsoft.com/api/stac/v1",
    modifier=planetary_computer.sign_inplace,)

# store bounding box
bbox = [-112.826843, 32.974108, -111.184387, 33.863574]

# search for bii data for area of interest
search = catalog.search(
    collections = ['io-biodiversity'],
    bbox = bbox,
    datetime = '2017/2020')

# retrieve items from search
items = search.item_collection()

# store items for years of interest
item_2020 = items[0]
item_2017 = items[3]

Information on Phoenix subdivisions will be created using US Census data:

In [33]:
Image(url=item_2020.assets['rendered_preview'].href, width=500)